# Part 4: Secure Model Serving Cloud Edition

Let's explore how easy it will be to modify Part 2 to support predictions in the cloud. We'll point out the differences below.

The first steps are to set up the cloud instances. There are few prerequisites that need to be setup prior to launching the cloud instances. This tutorial uses Google Cloud so you'll need an account set up there and need to have the gcloud command-line tool installed.

See [here](https://github.com/dropoutlabs/tf-world-tutorial/blob/master/private-prediction/CLOUD.md) for more details about installing the gcloud command-line tool and launching the instances.

Once that is completed you can continue with the tutorial.

In [1]:
from collections import OrderedDict

import numpy as np
import tensorflow as tf

import tf_encrypted as tfe
import tf_encrypted.keras.backend as KE

tf.compat.v1.disable_eager_execution()

## Model Setup

In [2]:
trained_model = '../saved_fl_model'
model = tf.keras.models.load_model(trained_model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Protocol

The only major change is here. We can just use the saved configuration file from setting up the cloud instances.

In [ ]:
config = tfe.RemoteConfig.load('/tmp/config.json')

In [ ]:
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

## Convert TF Keras into TFE Keras

Thanks to `tfe.keras.models.clone_model` you can convert automatically the TF Keras model into a TFE Keras model.

In [ ]:
with tfe.protocol.SecureNN():
    tfe_model = tfe.keras.models.clone_model(model)

## Set up a new `tfe.serving.QueueServer` 

`tfe.serving.QueueServer` will launch a serving queue, so that the TFE servers can accept prediction requests on the secured model from external clients.

In [ ]:
# Set up a new tfe.serving.QueueServer for the shared TFE model
q_input_shape = (1, 784)
q_output_shape = (1, 10)

server = tfe.serving.QueueServer(
    input_shape=q_input_shape, output_shape=q_output_shape, computation_fn=tfe_model
)

## Start Server

In [ ]:
sess = KE.get_session()

In [ ]:
request_ix = 1

def step_fn():
    global request_ix
    print("Served encrypted prediction {i} to client.".format(i=request_ix))
    request_ix += 1

server.run(
    sess,
    num_steps=3,
    step_fn=step_fn)

You are ready to move back to the **c - Private Prediction Client** notebook to request some private predictions. Make sure to restart the kernel and reset the cells if you've already run that notebook in the past.

## Cleanup!

Make sure to shut down your Google Cloud instances by following the instructions [here](TODO).